# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [6]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter
from datetime import datetime, timedelta

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [7]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets_renamed import POLYGON_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [32]:
def get_stock_price(symbol):
    base_url = "https://api.polygon.io/v2/aggs/ticker"
    endpoint = f"{base_url}/{symbol}/prev"
    params = {
        "adjusted": "true",
        "apiKey": POLYGON_API_TOKEN
    }
    response = requests.get(endpoint, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['resultsCount'] > 0:
            price = data['results'][0]['c']
            return data['results'][0]['c']
        else:
            print("No results found in data")
            return None
    else:
        print(f"Error: {response.status_code}")
        return None

In [31]:
def get_year_return(symbol):
    # Get current price
    current_price = get_stock_price(symbol)
    
    # Get year-ago price using a separate endpoint
    base_url = "https://api.polygon.io/v2/aggs/ticker"
    year_ago = (datetime.now() - timedelta(days=364)).strftime('%Y-%m-%d')
    endpoint = f"{base_url}/{symbol}/range/1/day/{year_ago}/{year_ago}"
    
    params = {
        "adjusted": "true",
        "apiKey": POLYGON_API_TOKEN
    }
    
    try:
        response = requests.get(endpoint, params=params)
        if response.status_code == 200:
            data = response.json()
            if data['resultsCount'] > 0:
                year_ago_price = data['results'][0]['c']
                year_returns = ((current_price - year_ago_price) / year_ago_price) * 100
                return year_returns
                
    except Exception as e:
        print(f"Error: {str(e)}")
    return None

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [33]:
g = get_year_return('AAPL')
print(g)

35.89171413471138


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [36]:
api_key = POLYGON_API_TOKEN
base_url = "https://api.polygon.io/v2/snapshot/locale/us/markets/stocks/tickers"
batch_api_call_url = f"{base_url}?apiKey={api_key}&tickers="

In [38]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
my_columns = ['Ticker', 'Price', 'One-Year Price Return (%)', 'Number of Shares to Buy']
stocks = stocks.sort_values('Ticker')

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns=my_columns)
missing_tickers = []  # To track tickers that don't return data

for symbol_string in symbol_strings:
    full_url = batch_api_call_url + symbol_string
    response = requests.get(full_url)
    
    if response.status_code == 200:
        data = response.json()
        if 'tickers' in data:
            # Keep track of which symbols we got data for
            received_symbols = set(td['ticker'] for td in data['tickers'])
            # Check which symbols we didn't get data for
            requested_symbols = set(symbol_string.split(','))
            missing_symbols = requested_symbols - received_symbols
            missing_tickers.extend(missing_symbols)
            
            for ticker_data in data['tickers']:
                symbol = ticker_data['ticker']
                price = ticker_data['day']['c'] if 'day' in ticker_data else None
                
                final_dataframe = pd.concat(
                    [
                        final_dataframe, 
                        pd.Series(
                            [
                                symbol,
                                price,
                                get_year_return(symbol),  # market cap
                                'N/A'
                            ],
                            index=my_columns
                        ).to_frame().T
                    ],
                    ignore_index=True
                )
    else:
        print(f"Error fetching data: {response.status_code}")

# Sort the final dataframe alphabetically by Ticker
final_dataframe = final_dataframe.sort_values('Ticker').reset_index(drop=True)
final_dataframe

,Ticker,Price,One-Year Price Return (%),Number of Shares to Buy
0,A,130.19,28.684393,N/A
1,AAL,13.15,17.620751,N/A
2,AAP,36.74,-28.423924,N/A
3,AAPL,231.41,35.891714,N/A
4,ABBV,187.85,32.391289,N/A
...,...,...,...,...
457,YUM,133.04,10.986902,N/A
458,ZBH,102.35,-1.025046,N/A
459,ZBRA,360.09,71.659437,N/A
460,ZION,51.34,71.247498,N/A


Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [59]:
# Sort the DataFrame by 'One-Year Price Return (%)' in descending order
final_dataframe = final_dataframe.sort_values('One-Year Price Return (%)', ascending=False)
# Select the top 50 rows
final_dataframe = final_dataframe.iloc[:50]
final_dataframe = final_dataframe.reset_index(drop=True)
final_dataframe = final_dataframe.drop('level_0', axis=1)
final_dataframe

,index,Ticker,Price,One-Year Price Return (%),Number of Shares to Buy
0,315,NVDA,141.54,243.869197,141
1,377,SLG,76.61,158.206943,261
2,356,RCL,200.91,133.997205,99
3,209,HWM,101.39,132.545872,197
4,34,ANET,394.09,124.271568,50
5,225,IRM,127.7,119.077029,156
6,430,VNO,43.17,114.562624,463
7,416,UAL,74.64,114.482759,267
8,409,TT,392.57,108.725011,50
9,310,NRG,87.53,107.171598,228


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [56]:
def num_of_shares(amount):
    position_size = amount/len(final_dataframe.index)
    for i in range(0, len(final_dataframe.index)):
        final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])

In [61]:
amount = input('What is the value of your portfolio')
amount = float(amount)
num_of_shares(amount)
final_dataframe

What is the value of your portfolio 1000000


,index,Ticker,Price,One-Year Price Return (%),Number of Shares to Buy
0,315,NVDA,141.54,243.869197,141
1,377,SLG,76.61,158.206943,261
2,356,RCL,200.91,133.997205,99
3,209,HWM,101.39,132.545872,197
4,34,ANET,394.09,124.271568,50
5,225,IRM,127.7,119.077029,156
6,430,VNO,43.17,114.562624,463
7,416,UAL,74.64,114.482759,267
8,409,TT,392.57,108.725011,50
9,310,NRG,87.53,107.171598,228


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [ ]:
writer = pd.ExcelWriter('recommended trade.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name="Recommended Trades", index=False)
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
    
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
    
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
    
)
writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}
for val in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{val}:{val}', 18, column_formats[val][1])
writer.close()